In [ ]:
import Parameters
import numpy as np
angle_antenna_user=40
a=10**(Parameters.G_BS_max/10)+np.max(np.max([-0.6*(angle_antenna_user/Parameters.bs_threedB)**2,-(Parameters.front_to_back_ratio/10)]))
print(Parameters.G_BS_max/10)
print(-0.6*(angle_antenna_user/Parameters.bs_threedB)**2)
print(-(Parameters.front_to_back_ratio/10))
print(np.max([-0.6*(angle_antenna_user/Parameters.bs_threedB)**2,-(Parameters.front_to_back_ratio/10)]))
print(a)

In [ ]:
import numpy as np

# 常量
R_earth = 6371000  # 地球半径，单位：米

# 输入数据
sate_position = [36.0735, 99.0898, 1134680.0]  # 卫星经纬度和海拔
user1_position = [37.33657329, 97.99781444, 0.0]  # 用户1经纬度和海拔
user2_position = [33.81267259, 98.5768581, 0.0]  # 用户2经纬度和海拔

# 转换为弧度制
def to_radians(coord):
    return [np.radians(coord[0]), np.radians(coord[1]), coord[2]]

sate_position_rad = to_radians(sate_position)
user1_position_rad = to_radians(user1_position)
user2_position_rad = to_radians(user2_position)

# 笛卡尔坐标转换
def spherical_to_cartesian(lat, lon, alt):
    x = (R_earth + alt) * np.cos(lat) * np.cos(lon)
    y = (R_earth + alt) * np.cos(lat) * np.sin(lon)
    z = (R_earth + alt) * np.sin(lat)
    return np.array([x, y, z])

sate_cartesian = spherical_to_cartesian(*sate_position_rad)
user1_cartesian = spherical_to_cartesian(*user1_position_rad)
user2_cartesian = spherical_to_cartesian(*user2_position_rad)

# 计算向量
vec_user1_to_sate = sate_cartesian - user1_cartesian
vec_user2_to_sate = sate_cartesian - user2_cartesian

# 计算夹角
def angle_between_vectors(v1, v2):
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(cos_theta))

angle = angle_between_vectors(vec_user1_to_sate, vec_user2_to_sate)

print(f"用户1与用户2关于卫星的夹角: {angle:.2f}°")


In [ ]:
import math
12.748060372140241*math.pi/180

In [ ]:

"""
计算两个用户相对于基站之间的夹角
user1, user2: 包含 'Lat', 'Lon', 'Alt' 的字典
base_station: 基站的 'Lat', 'Lon', 'Alt'
"""
# 将用户和基站的经纬度转换为三维坐标
base_station_pos = np.array([-1.6541e+06,6.62907e+06,3.19192e+06])
user1_pos = np.array([-1.61514e+06,5.2417e+06,3.24123e+06])
user2_pos = np.array([-1.36326e+06,5.1689e+06,3.46608e+06])

# 计算从基站到两个用户的向量
vector1 = user1_pos - base_station_pos
vector2 = user2_pos - base_station_pos

# 计算夹角（使用点积公式）
dot_product = np.dot(vector1, vector2)
magnitude1 = np.linalg.norm(vector1)
magnitude2 = np.linalg.norm(vector2)

cos_theta = dot_product / (magnitude1 * magnitude2)

# 计算夹角（单位: 度）
angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
angle

In [ ]:
import numpy as np
a=np.radians(np.array([-960266.85023803, 6001988.05446878, 4428082.36438297]))
B=np.radians(np.array([-791143.85252602, 5245523.60484765, 3552986.49571693]))
C=np.radians(np.array([-706449.31617299, 5028039.71670133, 3873086.44471723]))
print(a)
print(B)
print(C)

In [ ]:
import Satellite_Bs
import math
import Parameters
R=Parameters.R_earth
H=Parameters.HLeo
lla=[32.851528,98.238545,0.0]
u_xyz=Satellite_Bs.GeographicToCartesianCoordinates(lla[0],lla[1],lla[2],"GRS80")
sa_xyz=[-958408, 5.99098e+06, 4.41951e+06]
dis=math.sqrt(((u_xyz[0]-sa_xyz[0])**2)+((u_xyz[1]-sa_xyz[1])**2)+((u_xyz[2]-sa_xyz[2])**2))
ceta = ((R**2)+(dis**2)-((R+H)**2))/(2*R*dis)

angle =get_elevation_angle_geocentric(u_xyz,sa_xyz)
print(angle)

In [ ]:

import math

def get_elevation_angle_geocentric(g, s):
    """
    计算地面终端与卫星或高空平台之间的仰角
    g: 地面终端坐标 (x, y, z)
    s: 卫星或高空平台坐标 (x, y, z)
    返回值: 仰角（单位：度）
    """
    elev_angle = 0

    # 计算分子
    numerator = -g[0] * (s[0] - g[0]) - g[1] * (s[1] - g[1]) - g[2] * (s[2] - g[2])

    # 计算分母
    denominator = math.sqrt(g[0]**2 + g[1]**2 + g[2]**2) * math.sqrt((s[0] - g[0])**2 + (s[1] - g[1])**2 + (s[2] - g[2])**2)

    # 计算余弦值
    x = numerator / denominator

    # 防止 acos 输入值超出[-1, 1]范围
    if x > 1:
        x = 1

    # 计算仰角（将弧度转换为度并减去90度）
    elev_angle = (180.0 * math.acos(x)) / math.pi - 90

    # 确保仰角不是NaN
    if math.isnan(elev_angle):
        raise ValueError("acos returned a NaN value")

    return elev_angle

In [ ]:
def latlon_to_cartesian(lat, lon, alt=0):
    """
    将经纬度转换为三维地心坐标系中的坐标
    lat: 纬度 (单位: 度)
    lon: 经度 (单位: 度)
    alt: 高度 (单位: 米)
    """
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    R=6371000
    # 计算三维坐标
    X = (R + alt ) * np.cos(lat_rad) * np.cos(lon_rad)
    Y = (R + alt ) * np.cos(lat_rad) * np.sin(lon_rad)
    Z = (R + alt ) * np.sin(lat_rad)
    
    return np.array([X, Y, Z])

def angle_between_users(user1, user2, base_station):
    """
    计算两个用户相对于基站之间的夹角
    user1, user2: 包含 'Lat', 'Lon', 'Alt' 的字典
    base_station: 基站的 'Lat', 'Lon', 'Alt'
    """
    # 将用户和基站的经纬度转换为三维坐标
    base_station_pos = latlon_to_cartesian(base_station[0], base_station[1], base_station[2])
    user1_pos = latlon_to_cartesian(user1[0], user1[1], user1[2])
    user2_pos = latlon_to_cartesian(user2[0], user2[1], user2[2])
    
    # 计算从基站到两个用户的向量
    vector1 = user1_pos - base_station_pos
    vector2 = user2_pos - base_station_pos
    
    # 计算夹角（使用点积公式）
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    cos_theta = dot_product / (magnitude1 * magnitude2)
    
    # 计算夹角（单位: 度）
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
    
    return angle

In [ ]:
import math
math.log(62/40)

In [ ]:
user1=[0.68244101, 1.68533546, 0.        ]
user2=[0.66918577, 1.68430785, 0.        ]
a=[0.68535017, 1.68776165, 0.        ]
angle_between_users(user1,user2,a)


In [ ]:
data ={i: {
        "user_sa": [],
        "user_bs": [],
        "user_nonreq": []
    } for i in range(5)}

# 每个元素包含3个列表，分别命名为user_sa, user_bs, user_nonreq
data = [
    {"user_sa": [1, 2], "user_bs": [3, 4], "user_nonreq": [5, 6]},
    {"user_sa": [7, 8], "user_bs": [9, 10], "user_nonreq": [11, 12]},
    # 其他元素...
]

for idx,da in enumerate(data):
    
    



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
# 20个用户的经纬度和是否被服务 (纬度, 经度, 是否被服务)
user_data = np.array([
    [34.0, 97.0, 1],
    [34.1, 97.1, 0],
    [34.2, 97.2, 1],
    [34.3, 97.3, 1],
    [34.4, 97.4, 0],
    [34.5, 97.5, 0],
    [34.6, 97.6, 1],
    [34.7, 97.7, 1],
    [34.8, 97.8, 0],
    [34.9, 97.9, 0],
    [35.0, 98.0, 1],
    [35.1, 98.1, 0],
    [35.2, 98.2, 1],
    [35.3, 98.3, 0],
    [35.4, 98.4, 1],
    [35.5, 98.5, 1],
    [35.6, 98.6, 0],
    [35.7, 98.7, 0],
    [35.8, 98.8, 1],
    [35.9, 98.9, 1],
])

# 分离数据
latitudes = user_data[:, 0]  # 纬度
longitudes = user_data[:, 1]  # 经度
is_served = user_data[:, 2]  # 服务标志
user_ids = np.arange(1, len(user_data) + 1)  # 生成用户ID：1到20
# 设置颜色：被服务为蓝色，未被服务为红色
colors = ['blue' if served == 1 else 'red' for served in is_served]

# 绘图
plt.figure(figsize=(10, 6))
plt.scatter(longitudes, latitudes, c=colors, s=50, edgecolors='black', label='用户点')
for i, user_id in enumerate(user_ids):
    plt.text(longitudes[i], latitudes[i], f'{user_id}', fontsize=9, ha='right', va='bottom', color='black')
# 添加图例和标注
plt.title('用户分布及卫星服务状态', fontsize=14)
plt.xlabel('经度', fontsize=12)
plt.ylabel('纬度', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(['蓝色: 被卫星服务', '红色: 未被卫星服务'], loc='upper left')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from mpl_toolkits.basemap import Basemap  # 如果需要地理背景，可以使用 Basemap（可选）
import numpy as np

# 用户经纬度和服务情况
user_data = np.array([
    [34.201392, 97.152038, 1],  # [纬度, 经度, 服务状态]
    [34.342046, 97.383979, 0],
    [34.879392, 97.252038, 1],
    [34.601392, 97.882038, 0]
])
user_ids = np.arange(len(user_data))  # 用户ID

# 基站数据
base_latitude = 34.5
base_longitude = 97.5
radius = 50  # 覆盖范围（公里）

# 创建图
fig, ax = plt.subplots(figsize=(10, 8))

# 绘制用户点
for i, (lat, lon, served) in enumerate(user_data):
    color = 'blue' if served else 'red'
    ax.scatter(lon, lat, c=color, label='User' if i == 0 else "", s=50)
    ax.text(lon, lat, f"U{i}", fontsize=10, ha='center')

# 绘制基站位置
ax.scatter(base_longitude, base_latitude, c='green', label='Base Station', s=100, marker='^')

# 绘制基站覆盖范围（简单地假设地球是平面）
circle = Circle((base_longitude, base_latitude), radius / 111, color='green', alpha=0.2, label='Coverage Area')
ax.add_patch(circle)

# 设置图例和轴标签
ax.legend(loc='upper right')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('User Locations and Base Station Coverage')
ax.grid(True)

# 显示图像
plt.show()


In [ ]:
import torch
a=torch.tensor([[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
])

mask = a.repeat(1, 4).view(2,4,20)
print(mask)

In [6]:
import pandas as pd
data=[{'UserID': 0.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 34.785290025386054, 'Lon': 98.7413151204676, 'Alt': 0.0, 'Angle': 81.24053394996355, 'Dis_Bs': 31914.671986802918, 'ReqID': 1.0, 'QCI': 1.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 1.0, 'BsID': 0.0, 'BsIfServ': -1.0, 'Lat': 35.15065546865081, 'Lon': 98.22734982438702, 'Alt': 0.0, 'Angle': 82.29328455192496, 'Dis_Bs': 55098.83059692796, 'ReqID': 1.0, 'QCI': 3.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 2.0, 'BsID': 1.0, 'BsIfServ': -1.0, 'Lat': 37.23126170236018, 'Lon': 97.45731397852104, 'Alt': 0.0, 'Angle': 78.40457654882798, 'Dis_Bs': 61374.01102430176, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 4.0, 'BsID': 5.0, 'BsIfServ': -1.0, 'Lat': 34.92136988011389, 'Lon': 100.40123861519199, 'Alt': 0.0, 'Angle': 79.58348579498085, 'Dis_Bs': 55495.988924022466, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 5.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 34.885736621829274, 'Lon': 99.37091067617617, 'Alt': 0.0, 'Angle': 81.95932175340315, 'Dis_Bs': 28483.509989470353, 'ReqID': 1.0, 'QCI': 3.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 6.0, 'BsID': 5.0, 'BsIfServ': -1.0, 'Lat': 35.84127515797733, 'Lon': 99.87169227220511, 'Alt': 0.0, 'Angle': 85.49344723284017, 'Dis_Bs': 71439.37917950844, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 7.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 35.16811762953746, 'Lon': 99.25971964235319, 'Alt': 0.0, 'Angle': 83.90246001042749, 'Dis_Bs': 46391.22304624602, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 8.0, 'BsID': 3.0, 'BsIfServ': -1.0, 'Lat': 35.90887042392468, 'Lon': 98.86709007564863, 'Alt': 0.0, 'Angle': 88.38138088894041, 'Dis_Bs': 27202.016353718835, 'ReqID': 1.0, 'QCI': 2.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 9.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 34.496794771564346, 'Lon': 98.2180026508218, 'Alt': 0.0, 'Angle': 78.55839367386261, 'Dis_Bs': 85768.96203787295, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 10.0, 'BsID': 6.0, 'BsIfServ': -1.0, 'Lat': 36.61068431036685, 'Lon': 100.8116067132173, 'Alt': 0.0, 'Angle': 80.10294337844448, 'Dis_Bs': 31360.04926810224, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 12.0, 'BsID': 5.0, 'BsIfServ': -1.0, 'Lat': 35.69989618417633, 'Lon': 100.778380761951, 'Alt': 0.0, 'Angle': 80.56950860988422, 'Dis_Bs': 42598.567301937816, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 13.0, 'BsID': 0.0, 'BsIfServ': -1.0, 'Lat': 35.184714050764406, 'Lon': 97.19734026031259, 'Alt': 0.0, 'Angle': 78.2485549807856, 'Dis_Bs': 53872.837039728794, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 14.0, 'BsID': 1.0, 'BsIfServ': 1.0, 'Lat': 37.26642752307291, 'Lon': 98.1619353974462, 'Alt': 0.0, 'Angle': 80.63179792283583, 'Dis_Bs': 74315.86863243717, 'ReqID': 1.0, 'QCI': 2.0, 'NewData': 5000000.0, 'Last_WaitData': 4377612.054100811, 'Total_WaitData': 5038083.56635245, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 4339528.48774836, 'Down_Throughput': 4138.496864078866}, {'UserID': 15.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 35.09669704858783, 'Lon': 99.65989611219085, 'Alt': 0.0, 'Angle': 82.80392980024777, 'Dis_Bs': 63177.49077416021, 'ReqID': 1.0, 'QCI': 2.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 16.0, 'BsID': 3.0, 'BsIfServ': -1.0, 'Lat': 35.99026495511102, 'Lon': 99.76456459704663, 'Alt': 0.0, 'Angle': 86.30453278442369, 'Dis_Bs': 61517.82805933904, 'ReqID': 1.0, 'QCI': 2.0, 'NewData': 5000000.0, 'Last_WaitData': 0.0, 'Total_WaitData': 5000000.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 18.0, 'BsID': 2.0, 'BsIfServ': -1.0, 'Lat': 34.92794528285915, 'Lon': 98.52365597220154, 'Alt': 0.0, 'Angle': 81.79509015334574, 'Dis_Bs': 54480.039791553376, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}, {'UserID': 19.0, 'BsID': 0.0, 'BsIfServ': -1.0, 'Lat': 35.739652847280276, 'Lon': 98.15278763428736, 'Alt': 0.0, 'Angle': 84.47210174706422, 'Dis_Bs': 53435.55854114415, 'ReqID': 0.0, 'QCI': 0.0, 'NewData': 0.0, 'Last_WaitData': 0.0, 'Total_WaitData': 0.0, 'Finish_Data': 0.0, 'Time_Delay': 0.0, 'Down_TxData': 0.0, 'Down_Throughput': 0.0}]
df = pd.DataFrame(data)

# 指定列名，计算其总和
columns_to_sum = ['NewData', 'Total_WaitData', 'Dis_Bs']  # 需要求和的列
sum_result = df[columns_to_sum].sum()
column_names = df.columns
print(column_names)
# 打印结果
print(sum_result)

Index(['UserID', 'BsID', 'BsIfServ', 'Lat', 'Lon', 'Alt', 'Angle', 'Dis_Bs',
       'ReqID', 'QCI', 'NewData', 'Last_WaitData', 'Total_WaitData',
       'Finish_Data', 'Time_Delay', 'Down_TxData', 'Down_Throughput'],
      dtype='object')
NewData           3.500000e+07
Total_WaitData    3.503808e+07
Dis_Bs            8.979268e+05
dtype: float64


In [ ]:
import cvxpy as cp
import numpy as np

def power_optimization(SINR, h, Pmax, B, T_b, phi_max, Delta_s):
    """
    Solve the power optimization problem.

    Parameters:
    SINR (np.ndarray): Array of SINR values (size N_b).
    h (np.ndarray): Channel gain matrix (size N_b x N_b).
    Pmax (float): Maximum total power constraint.
    B (float): Bandwidth.
    T_b (float): Time block duration.
    phi_max (float): Maximum Shannon capacity.
    Delta_s (float): Data block size.

    Returns:
    np.ndarray: Optimized power allocation.
    """
    Nb = len(SINR)  # Number of base stations

    # Compute alpha based on the given formula
    alpha = (SINR ** 2) / (1 + SINR)

    # Define variables
    P = cp.Variable(Nb, nonneg=True)  # Power allocation for each base station
    SINR_var = cp.Variable(Nb, nonneg=True)  # Auxiliary variables for SINR
    z = cp.Variable(Nb)  # Auxiliary variables for log approximation

    # Define the objective function
    objective = cp.Minimize(cp.sum(-alpha / SINR_var))

    # Define constraints
    constraints = []

    # C6: Total power constraint
    constraints.append(cp.sum(P) <= Pmax)

    # C7: Reformulate SINR and capacity constraints
    for k in range(Nb):
        interference = cp.sum(h[:, k] * P) - h[k, k] * P[k]
        constraints.append(h[k, k] * P[k] >= SINR_var[k] * (interference + 1e-9))  # Reformulated SINR
        constraints.append(1 + SINR_var[k] >= cp.exp(z[k]))  # Reformulated log constraint
        constraints.append(z[k] <= (phi_max * Delta_s) / (B * T_b))  # Capacity limit

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if problem.status == cp.OPTIMAL:
        return P.value
    else:
        raise ValueError("Optimization problem could not be solved.")

# Example usage
# Inputs (example values)
SINR = np.array([10, 15, 20])
h = np.array([
    [1.0, 0.1, 0.2],
    [0.1, 1.0, 0.3],
    [0.2, 0.3, 1.0]
])
Pmax = 10  # Maximum total power
B = 1e6  # Bandwidth in Hz
T_b = 1e-3  # Time block in seconds
phi_max = 1e3  # Maximum Shannon capacity
Delta_s = 1  # Data block size

# Solve optimization problem
try:
    P_optimized = power_optimization(SINR, h, Pmax, B, T_b, phi_max, Delta_s)
    print("Optimized Power Allocation:", P_optimized)
except ValueError as e:
    print(str(e))


#### 梯度下降法

In [9]:
import numpy as np
import scipy.optimize as opt

# 已知参数（示例数据，需要根据具体情况替换）
N_b = 6  # 用户数
P_max = 30  # 最大总功率

# alpha、SINR和信道增益矩阵的示例数据
SINR = np.random.rand(N_b)   # 用户k的SINR值
h = np.random.rand(N_b, N_b)  # 信道增益矩阵，假设为对称矩阵
P_initial = np.ones(N_b) * 10  # 初始功率值（可以根据实际情况调整）

# Shannon容量约束中的B和T_b（假设这些值已知）
B = 500e6  # 带宽
T_b = 1e-3  # 每个用户的时间间隔
phi_max = 1e3  # 最大容量约束
Delta_s = 1  # 时间窗

def objective(P):
    """ 目标函数：最小化功率（负的SINR） """
    P = np.maximum(P, 0)  # 确保功率非负
    SINR_k = SINR  # 使用给定的SINR
    alpha_k = (SINR ** 2) / (1 + SINR)  # 使用给定的alpha
    return -np.sum(alpha_k / SINR_k)  # 返回负的目标值

def constraints(P):
    """ 定义约束条件 """
    cons = []

    # 非负功率约束
    cons.append({'type': 'ineq', 'fun': lambda P: P})  # P_k >= 0
    # 总功率约束
    cons.append({'type': 'ineq', 'fun': lambda P: P_max - np.sum(P)})  # 总功率 <= P_max

    # Shannon容量约束
    for k in range(N_b):
        def shannon_constraint(P):
            # 计算Shannon容量约束
            interference = np.sum(h[k, :] * P) - h[k, k] * P[k]
            capacity = B * T_b * np.log2(1 + (h[k, k] * P[k]) / interference)
            return phi_max * Delta_s - capacity  # 返回容量约束
    
        cons.append({'type': 'ineq', 'fun': shannon_constraint})
        
    return cons


# 初始功率值
P_initial = np.ones(N_b) * 10  # 初始功率值

# 调用优化函数进行求解
result = opt.minimize(objective, P_initial, constraints=constraints(P_initial), bounds=[(0, P_max)]*N_b, method='SLSQP')

# 输出优化结果
if result.success:
    print("Optimal power distribution:", result.x)
else:
    print("Optimization failed:", result.message)


Optimal power distribution: [7.99360578e-15 9.80320053e+00 9.00010927e-15 5.02044240e+00
 1.51763571e+01 1.13006162e-14]


#### 拉格朗日乘子法

In [10]:
import numpy as np
from scipy.optimize import fsolve

# 已知参数（示例数据，需要根据具体情况替换）
N_b = 6  # 用户数
P_max = 30  # 最大总功率

# alpha、SINR和信道增益矩阵的示例数据
SINR = np.array([1, 1, 2, 5, 10, 20])   # 用户k的SINR值
print("SINR:", SINR)
alpha = (SINR ** 2) / (1 + SINR)  # 使用给定的alpha
h = np.ones((N_b, N_b))  # 信道增益矩阵，假设为对称矩阵
print("Channel gain matrix:", h)
P_initial = np.ones(N_b) * (P_max/N_b)  # 初始功率值

# Shannon容量约束中的B和T_b（假设这些值已知）
B = 500e6  # 带宽
T_b = 1e-3  # 每个用户的时间间隔
phi_max = 1e3  # 最大容量约束
Delta_s = 1  # 时间窗

# 定义目标函数和约束
def lagrangian_multiplier(P, lambd, mu):
    """ 拉格朗日函数 """
    lagrangian = -np.sum(alpha / SINR) + lambd * (np.sum(P) - P_max) + np.sum(mu * P)
    return lagrangian

def kkt_conditions(P):
    """ KKT 条件求解 """
    lambd = np.random.rand()  # 初始化拉格朗日乘子
    mu = np.random.rand(N_b)  # 初始化非负约束的拉格朗日乘子

    # 构造一阶条件
    def equations(vars):
        P = vars[:N_b]
        lambd = vars[N_b]
        mu = vars[N_b+1:]

        eqs = []

        # 梯度条件
        for k in range(N_b):
            eqs.append(-alpha[k] / SINR[k] + lambd - mu[k])

        # 总功率约束
        eqs.append(np.sum(P) - P_max)

        # 非负功率乘子条件
        for k in range(N_b):
            eqs.append(mu[k] * P[k])

        return eqs

    # 求解非线性方程组
    solution = fsolve(equations, np.ones(N_b + 1 + N_b))
    return solution[:N_b]  # 返回功率分配结果

P_optimal = kkt_conditions(np.random.rand(N_b))
print("Optimal Power Allocation (Lagrangian Method):", P_optimal)


SINR: [ 1  1  2  5 10 20]
Channel gain matrix: [[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
Optimal Power Allocation (Lagrangian Method): [-1.75073049e-11 -1.75073050e-11  1.54791376e-10 -9.59566909e-10
  3.00000000e+01 -1.09370449e-08]


In [ ]:
import cvxpy as cp
import numpy as np

# 已知参数
N_b = 10
P_max = 100
alpha = np.random.rand(N_b)
SINR = np.random.rand(N_b)
h = np.random.rand(N_b, N_b)
B = 1
T_b = 1
phi_max = 1
Delta_s = 1

# 定义变量
P = cp.Variable(N_b)  # 每个用户的功率分配

# 定义目标函数
objective = cp.Minimize(-cp.sum(alpha / SINR))  # 最小化负的目标函数

# 定义约束
constraints = [
    P >= 0,                           # 非负功率约束
    cp.sum(P) <= P_max                # 总功率约束
]

# Shannon容量约束
for k in range(N_b):
    interference = cp.sum(h[k, :] * P) - h[k, k] * P[k]
    SINR_k = h[k, k] * P[k] / (interference + 1e-9)  # 防止零除
    constraints.append(B * T_b * cp.log(1 + SINR_k) <= phi_max * Delta_s)

# 解决优化问题
problem = cp.Problem(objective, constraints)
problem.solve()

# 输出结果
if problem.status == cp.OPTIMAL:
    print("Optimal Power Allocation (Convex Optimization):", P.value)
else:
    print("Optimization failed:", problem.status)


In [1]:
import torch
from torch.distributions.categorical import Categorical

dis = torch.tensor([0.4, 0.4, 0.2])
print('dis:', dis)
dist = Categorical(probs = dis)
print('dist:', dist.probs)
dis[2] = 0
print('dis:', dis)
dist = Categorical(probs = dis)
print('dist:', dist.probs)


dis: tensor([0.4000, 0.4000, 0.2000])
dist: tensor([0.4000, 0.4000, 0.2000])
dis: tensor([0.4000, 0.4000, 0.0000])
dist: tensor([0.5000, 0.5000, 0.0000])
